In [2]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
import zipfile
import io
import requests

# Load the dataset
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# Extract necessary files
zip_file.extractall('ml-latest-small')

# Load the data into DataFrames
ratings = pd.read_csv('ml-latest-small/ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/ml-latest-small/movies.csv')

# Create user-item interaction matrix
user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Convert to numpy array
R = user_item_matrix.values
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

# Perform SVD
U, sigma, Vt = svds(R_demeaned, k=50)
sigma = np.diag(sigma)

# Predict ratings
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns=user_item_matrix.columns)

# Recommend movies to a specific user
def recommend_movies(preds_df, user_id, movies_df, original_ratings_df, num_recommendations=5):
    user_row_number = user_id - 1  # User ID starts at 1 in the dataset
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)

    user_data = original_ratings_df[original_ratings_df.userId == user_id]
    user_full = (user_data.merge(movies_df, how='left', left_on='movieId', right_on='movieId').
                 sort_values(['rating'], ascending=False))

    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
                        merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                              left_on='movieId', right_on='movieId').
                        rename(columns={user_row_number: 'Predictions'}).
                        sort_values('Predictions', ascending=False).
                        iloc[:num_recommendations, :-1])

    return user_full, recommendations

# Example usage
user_id = 1
user_full, recommendations = recommend_movies(preds_df, user_id, movies, ratings, num_recommendations=5)

print(f"User {user_id} has already rated:")
print(user_full.head())

print("\nTop 5 movie recommendations:")
print(recommendations)


User 1 has already rated:
     userId  movieId  rating  timestamp  \
231       1     5060     5.0  964984002   
185       1     2872     5.0  964981680   
89        1     1291     5.0  964981909   
90        1     1298     5.0  964984086   
190       1     2948     5.0  964982191   

                                         title                     genres  
231               M*A*S*H (a.k.a. MASH) (1970)           Comedy|Drama|War  
185                           Excalibur (1981)          Adventure|Fantasy  
89   Indiana Jones and the Last Crusade (1989)           Action|Adventure  
90                 Pink Floyd: The Wall (1982)              Drama|Musical  
190               From Russia with Love (1963)  Action|Adventure|Thriller  

Top 5 movie recommendations:
      movieId                           title                 genres
736      1036                 Die Hard (1988)  Action|Crime|Thriller
844      1221  Godfather: Part II, The (1974)            Crime|Drama
974      1387         